# Bluesky

- A collection of Python libraries for analysis-friendly data collection
- Also, confusingly, the name of one of those libraries 
- Developed by NSLS-II since 2015


### Useful links:
- [Github](https://github.com/bluesky/)
- [High Level Documentation](https://nsls-ii.github.io/)
- [Tutorial](https://nsls-ii.github.io/bluesky/tutorial.html)
- [DLS Demo on P47 Test Rig](https://dlsltd-my.sharepoint.com/:f:/g/personal/callum_forrester_diamond_ac_uk/EoCVYM1VtpdHrltBQYQ7ePkBYhyhRSt_pmJsDxpHEcoF5Q?e=MOCEdX)


## Components

![architecture](block-diagram.png "Architecture")

Decouples a number of components, separated by well-defined interfaces. You can swap out any of these with a new library obeying the interfaces.

| Library     | Purpose                                                | Analogous DLS Components              | Github                                   |
|-------------|--------------------------------------------------------|---------------------------------------|------------------------------------------|
| Ophyd       | Device Management                                      | GDA Scannables, Devices, Detectors    | https://github.com/bluesky/ophyd         |
| Bluesky     | Orchestration, Sequencing, Exporting Collection Events | GDA Scannables, Scans, Jython Scripts | https://github.com/bluesky/bluesky       |
| DataBroker  | Event Collection/Aggregation, I/O-agnostic Data Access | Dawn Data Server, NeXus Files         | https://github.com/bluesky/databroker    |
| Suitcase    | Data Export Plugins for DataBroker                     | GDA Nexus Writer                      | https://github.com/bluesky/suitcase-core |
| Event Model | Formal Structure for Events                            | Possibly the DIAD acquisition model   | https://github.com/bluesky/event-model   |


## Interfaces

Documentation: https://nsls-ii.github.io/ophyd/architecture.html

![uml](uml.png "UML")

Note, Python typing means NSLS-II are now foramlly defining these interfaces in the code, see PR [#1446](https://github.com/bluesky/bluesky/pull/1446).


## Data

- Each `Readable` device exports data as JSON, it can also export references to data, for example a detector can export a reference to an HDF5 file.
- The `RunEngine` exports events (conforming to the Event Model) that include this data, plus various metadata.
- DataBroker captures and aggregates these events for later access. 
- Optionally, any suitcase plugins for DataBroker will write the data to the user's desired format (NSLS-II are not NeXus fanatics like us). 


## Plans

- Recipes for sequences of actions (which usually export events)
- Normally written as Python "yield" functions but can be any iterable of Bluesky `Msg`s.



## The `RunEngine`

In [ ]:
from bluesky import RunEngine

RE = RunEngine()

# Simplest possible plan:
RE([])

In [ ]:
from bluesky import Msg
from pprint import pprint

from typing import Dict, Any


def show_events(name: str, doc: Dict[str, Any]) -> None:
    pprint({name: doc})


RE([Msg("open_run"), Msg("close_run")], show_events)

## The Yield Syntax

In [ ]:
def my_plan():
    yield Msg("open_run")
    yield Msg("close_run")

# Lazy generator
my_plan()

In [ ]:
# Evaulates to the same as the plan above
list(my_plan())

In [ ]:
# Produces same behavoir from RunEngine
RE(my_plan(), show_events)

## Example Plans

In [ ]:
import bluesky.plan_stubs as bps
import bluesky.plans as bp

In [ ]:
def empty_run():
    yield from bps.open_run()
    yield from bps.sleep(3)
    yield from bps.close_run()

RE(empty_run(), show_events)

## Devices

In [ ]:
import os
import socket
from ophyd import EpicsMotor

# Device wrapping ADSIM motor, need to set EPICS dummy ports and use machine hostname
os.environ["EPICS_CA_SERVER_PORT"] = "6064"
os.environ["EPICS_CA_REPEATER_PORT"] = "6065"
hostname = socket.gethostname()


x = EpicsMotor(name="x", prefix=f"{hostname}-MO-SIM-01:M1")

In [ ]:
x.wait_for_connection()
x

In [ ]:
x.read()

In [ ]:
x.set(x.position + 1).wait()
x.read()

### Device Heirarchy

In [ ]:
from ophyd import MotorBundle, Component as Cpt

class SimBundle(MotorBundle):
    """
    ADSIM EPICS motors
    """

    x: EpicsMotor = Cpt(EpicsMotor, "M1")
    y: EpicsMotor = Cpt(EpicsMotor, "M2")
    z: EpicsMotor = Cpt(EpicsMotor, "M3")
    theta: EpicsMotor = Cpt(EpicsMotor, "M4")
    load: EpicsMotor = Cpt(EpicsMotor, "M5")

motors = SimBundle(name="motors", prefix=f"{hostname}-MO-SIM-01:")
motors.wait_for_connection()

[component.dotted_name for component in motors.walk_components()]

### Moving Motors

In [ ]:
RE(bps.mvr(motors.x, 1))

In [ ]:
RE(bps.mvr(
    motors.x, 1,
    motors.y, -1
))

### Detector

- Tree of components, top level components are the driver and plugins

In [ ]:
from ophyd import DetectorBase, SingleTrigger
from ophyd.areadetector.cam import AreaDetectorCam
from ophyd.areadetector.plugins import HDF5Plugin, PosPlugin, StatsPlugin
from ophyd.areadetector.filestore_mixins import FileStoreHDF5, FileStoreIterativeWrite

DATA_ROOT: str = "/tmp"
DATA_WRITE_PATH_TEMPLATE: str = "%Y"


class AdSimDetector(SingleTrigger, DetectorBase):
    class HDF5File(HDF5Plugin, FileStoreHDF5, FileStoreIterativeWrite):
        pool_max_buffers = None
        file_number_sync = None
        file_number_write = None

        def get_frames_per_point(self):
            return self.parent.cam.num_images.get()

    cam: AreaDetectorCam = Cpt(AreaDetectorCam, suffix="CAM:")
    stat: StatsPlugin = Cpt(StatsPlugin, suffix="STAT:")
    pos: PosPlugin = Cpt(PosPlugin, suffix="POS:")
    hdf: HDF5File = Cpt(
        HDF5File,
        suffix="HDF5:",
        root=DATA_ROOT,
        write_path_template=DATA_WRITE_PATH_TEMPLATE,
    )

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.hdf.kind = "normal"

        self.stage_sigs = {
            # Setup the plugin chain
            self.stat.nd_array_port: self.cam.port_name.get(),
            self.hdf.nd_array_port: self.cam.port_name.get(),
            
            # Setup Driver
            self.cam.array_counter: 0,
            self.cam.image_mode: "Multiple",
            self.cam.trigger_mode: "Internal",
            
            **self.stage_sigs
        }

    @property
    def hints(self) -> Dict[str, Any]:
        return {"fields": "sum"}
    
    def describe(self) -> Dict[str, Any]:
        return {
            "sum": {
                "source": self.stat.sum.name,
                "dtype": "number",
                "shape": [],
                "units": "count"
            }
            **super().describe()
        }

    def read(self) -> Dict[str, Any]:
        return {
            "sum": self.stat.sum.value,
            **super().read()
        }


In [ ]:
det = AdSimDetector(name="adsim", prefix=f"{hostname}-AD-SIM-01:")
det.wait_for_connection()

# Make sure the plugin is primed
#det.hdf.warmup()

# Set transient values to how we want them
RE(bps.mv(
    det.cam.num_images, 1, 
    det.cam.acquire_time, 0.1, 
    det.cam.acquire_period, 0.11))


# Give it a directory to write data
if not os.path.exists(f"{DATA_ROOT}/2021"):
    os.mkdir(f"{DATA_ROOT}/2021")

In [ ]:
RE(bp.count([det]), show_events)